In [45]:
#Library Imports
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() # tensorflow 2의 기능은 빼고

# 0. 계획
일단 bAbI 전체 파일은 git clone으로 가져다놓은 상태.  
수업때 다루었던 6번 파일의 train 데이터로 먼저 가공하는 법을 익혀보자.

# 1. 데이터 가져오기
Github에 올라와있는 bAbI 데이터는 txt로 되어있다.  
pandas로 읽어오려 했지만 추후의 데이터 처리가 복잡해질 것 같아 Python 기본 file system으로 가져오기로 함.

In [24]:
raw_data = open('../../bAbI/tasks_1-20_v1-2/en/qa6_yes-no-questions_train.txt', 'r').readlines()
raw_data

['1 Mary moved to the bathroom.\n',
 '2 Sandra journeyed to the bedroom.\n',
 '3 Is Sandra in the hallway? \tno\t2\n',
 '4 Mary went back to the bedroom.\n',
 '5 Daniel went back to the hallway.\n',
 '6 Is Daniel in the bathroom? \tno\t5\n',
 '7 Sandra went to the kitchen.\n',
 '8 Daniel went back to the bathroom.\n',
 '9 Is Daniel in the office? \tno\t8\n',
 '10 Daniel picked up the football there.\n',
 '11 Daniel went to the bedroom.\n',
 '12 Is Daniel in the bedroom? \tyes\t11\n',
 '13 John travelled to the office.\n',
 '14 Sandra went to the garden.\n',
 '15 Is Daniel in the bedroom? \tyes\t11\n',
 '1 Sandra got the football there.\n',
 '2 Mary went to the bedroom.\n',
 '3 Is Mary in the bedroom? \tyes\t2\n',
 '4 Daniel got the apple there.\n',
 '5 Sandra travelled to the hallway.\n',
 '6 Is Sandra in the office? \tno\t5\n',
 '7 Sandra moved to the garden.\n',
 '8 Mary travelled to the kitchen.\n',
 '9 Is Sandra in the bathroom? \tno\t7\n',
 '10 Sandra went back to the bedroom.\n',

데이터를 불러와보니 각 행 처음에는 줄 번호, 마지막에는 줄바꿈(\n)이 포함되어있다.  
텍스트 인덱싱으로 없애주자.

In [25]:
raw_data = [row[2:-1].strip() for row in raw_data]
raw_data

['Mary moved to the bathroom.',
 'Sandra journeyed to the bedroom.',
 'Is Sandra in the hallway? \tno\t2',
 'Mary went back to the bedroom.',
 'Daniel went back to the hallway.',
 'Is Daniel in the bathroom? \tno\t5',
 'Sandra went to the kitchen.',
 'Daniel went back to the bathroom.',
 'Is Daniel in the office? \tno\t8',
 'Daniel picked up the football there.',
 'Daniel went to the bedroom.',
 'Is Daniel in the bedroom? \tyes\t11',
 'John travelled to the office.',
 'Sandra went to the garden.',
 'Is Daniel in the bedroom? \tyes\t11',
 'Sandra got the football there.',
 'Mary went to the bedroom.',
 'Is Mary in the bedroom? \tyes\t2',
 'Daniel got the apple there.',
 'Sandra travelled to the hallway.',
 'Is Sandra in the office? \tno\t5',
 'Sandra moved to the garden.',
 'Mary travelled to the kitchen.',
 'Is Sandra in the bathroom? \tno\t7',
 'Sandra went back to the bedroom.',
 'Daniel put down the apple.',
 'Is Sandra in the bathroom? \tno\t10',
 'Sandra put down the football.',
 

이제 데이터를 각 행이 '지문/질문/정답'의 세 성분을 가지도록 구성해야 한다.  
각 성분의 구분은 \t를 기준으로 할 예정.

In [118]:
for i in range(5):
    if '\t' in raw_data[i]: txt = 'True'
    else: txt = 'False'
    print(i, 'row: ', raw_data[i],": ", txt)

0 row:  Mary moved to the bathroom. :  False
1 row:  Sandra journeyed to the bedroom. :  False
2 row:  Is Sandra in the hallway? 	no	2 :  True
3 row:  Mary went back to the bedroom. :  False
4 row:  Daniel went back to the hallway. :  False


\t가 들어간 행에는 질문과 답, 그리고 답의 근거가 되는 행 번호가 들어있다.  
행 번호는 당장은 필요 없기 때문에 데이터를 조정하는 과정에서 빼주기로 한다.

In [5]:
# punc_mark: input list의 마지막 원소에 .나 ?가 있을 시 이를 분리해줌
def punc_mark(input):
    if '.' in input[-1] or '?' in input[-1]:
        return input[:-1] + [input[-1][:-1], input[-1][-1]]
    else:
        return input

In [48]:
# train_data가 우리의 목표 데이터
train_data = []

prev = 0
curr = 0
temp = False
for i in range(len(raw_data)):
    # 만약 i번째 행이 \t를 가지고 있으면
    if '\t' in raw_data[i]:
        # i번째 행까지를 new_row로 빼고 raw_data를 업데이트
        prev, curr = curr, i+1
        new_row = raw_data[prev:curr]
        # new_row의 마지막 행을 \t를 기준으로 나눠주고
        # 마지막에 있는 행 번호를 지워준다
        new_row = new_row[:-1] + new_row[-1].split('\t')[:-1]
        new_text = [item for sublist in [punc_mark(item) for item in [item.split(' ') for item in new_row[:-2]]] for item in sublist]
        new_question = punc_mark([item for item in new_row[-2].split(' ') if item != ''])
        new_answer = new_row[-1]
        train_data.append([new_text, new_question, new_answer])

In [50]:
train_data[:10]

[[['Mary',
   'moved',
   'to',
   'the',
   'bathroom',
   '.',
   'Sandra',
   'journeyed',
   'to',
   'the',
   'bedroom',
   '.'],
  ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
  'no'],
 [['Mary',
   'went',
   'back',
   'to',
   'the',
   'bedroom',
   '.',
   'Daniel',
   'went',
   'back',
   'to',
   'the',
   'hallway',
   '.'],
  ['Is', 'Daniel', 'in', 'the', 'bathroom', '?'],
  'no'],
 [['Sandra',
   'went',
   'to',
   'the',
   'kitchen',
   '.',
   'Daniel',
   'went',
   'back',
   'to',
   'the',
   'bathroom',
   '.'],
  ['Is', 'Daniel', 'in', 'the', 'office', '?'],
  'no'],
 [['Daniel',
   'picked',
   'up',
   'the',
   'football',
   'there',
   '.',
   'Daniel',
   'went',
   'to',
   'the',
   'bedroom',
   '.'],
  ['Is', 'Daniel', 'in', 'the', 'bedroom', '?'],
  'yes'],
 [['John',
   'travelled',
   'to',
   'the',
   'office',
   '.',
   'Sandra',
   'went',
   'to',
   'the',
   'garden',
   '.'],
  ['Is', 'Daniel', 'in', 'the', 'bedroom', '?'],
  'yes'],


In [51]:
df = pd.DataFrame(train_data, columns = ('text','question','answer'))
df

,text,question,answer
0,"[Mary, moved, to, the, bathroom, ., Sandra, jo...","[Is, Sandra, in, the, hallway, ?]",no
1,"[Mary, went, back, to, the, bedroom, ., Daniel...","[Is, Daniel, in, the, bathroom, ?]",no
2,"[Sandra, went, to, the, kitchen, ., Daniel, we...","[Is, Daniel, in, the, office, ?]",no
3,"[Daniel, picked, up, the, football, there, ., ...","[Is, Daniel, in, the, bedroom, ?]",yes
4,"[John, travelled, to, the, office, ., Sandra, ...","[Is, Daniel, in, the, bedroom, ?]",yes
...,...,...,...
995,"[Mary, went, to, the, bathroom, ., John, took,...","[Is, Mary, in, the, hallway, ?]",no
996,"[Sandra, moved, to, the, office, ., John, drop...","[Is, Mary, in, the, bathroom, ?]",yes
997,"[Mary, went, to, the, office, ., Daniel, journ...","[Is, Sandra, in, the, office, ?]",yes
998,"[Mary, journeyed, to, the, hallway, ., Sandra,...","[Is, Mary, in, the, office, ?]",no


# 2. 단어 벡터 만들기

먼저 모든 단어를 모은 집합을 만들자.

In [52]:
voca = set()
for text, question, answer in train_data:
    voca = voca.union(set(text))
    voca = voca.union(set(question))
    voca.add(answer)

In [53]:
len(voca)+1

38

keras에서 필요한 기능들을 가져오자.  
`Tokenizer`는 토큰화시켜주는 기능,  
`pad_sequences`는 길이를 일정하게 맞춰주는 기능을 한다.

In [76]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

단어집합을 토큰화시키고 인덱스를 확인.

In [77]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(voca)
tokenizer.word_index

{'apple': 1,
 'took': 2,
 '.': 3,
 'picked': 4,
 'moved': 5,
 'john': 6,
 'garden': 7,
 'put': 8,
 'mary': 9,
 'grabbed': 10,
 'yes': 11,
 'kitchen': 12,
 'back': 13,
 'up': 14,
 'bathroom': 15,
 'bedroom': 16,
 'milk': 17,
 'sandra': 18,
 '?': 19,
 'the': 20,
 'journeyed': 21,
 'there': 22,
 'in': 23,
 'football': 24,
 'got': 25,
 'daniel': 26,
 'hallway': 27,
 'discarded': 28,
 'dropped': 29,
 'office': 30,
 'down': 31,
 'is': 32,
 'left': 33,
 'travelled': 34,
 'no': 35,
 'to': 36,
 'went': 37}

In [78]:
max_text_len = max([len(data[0]) for data in train_data])
max_question_len = max([len(data[1]) for data in train_data])

In [80]:
train_text_seq = pad_sequences(tokenizer.texts_to_sequences(df['text']), maxlen = max_text_len)
train_question_seq = pad_sequences(tokenizer.texts_to_sequences(df['question']), maxlen = max_question_len)
train_answer_seq = tokenizer.texts_to_sequences(df['answer'])

In [83]:
train_text_seq[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  5, 36, 20, 15,  3, 18,
       21, 36, 20, 16,  3])

# 3. 모델 구성

In [85]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

placeholder를 `Input`을 통해 만든다.  
인자로 길이와 batch_size를 받지만, 아직 모르니 비워놓는다.

In [86]:
input_sequence = Input((max_text_len,))
question = Input((max_question_len,))

## 3.1. Embedding
단어를 dense vector로 만드는 작업. 인공 신경망 용어로는 embedding layer를 만드는 역할.

```python
help(tf.keras.layers.Embedding)
```
```
Help on class Embedding in module tensorflow.python.keras.layers.embeddings:

class Embedding(tensorflow.python.keras.engine.base_layer.Layer)
 |  Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs)
 |  
 |  Turns positive integers (indexes) into dense vectors of fixed size.
 |  
 |  e.g. `[[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]`
 |  
 |  This layer can only be used as the first layer in a model.
 |  
 |  Example:
 |  
 |  ~~~python
 |  model = Sequential()
 |  model.add(Embedding(1000, 64, input_length=10))
 |  # the model will take as input an integer matrix of size (batch, input_length).
 |  # the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
 |  # now model.output_shape == (None, 10, 64), where None is the batch dimension.
 |  
 |  input_array = np.random.randint(1000, size=(32, 10))
 |  
 |  model.compile('rmsprop', 'mse')
 |  output_array = model.predict(input_array)
 |  assert output_array.shape == (32, 10, 64)
 |  ~~~
 |  
 |  Arguments:
 |    input_dim: int > 0. Size of the vocabulary,
 |      i.e. maximum integer index + 1.
 |    output_dim: int >= 0. Dimension of the dense embedding.
 |    embeddings_initializer: Initializer for the `embeddings` matrix.
 |    embeddings_regularizer: Regularizer function applied to
 |      the `embeddings` matrix.
 |    embeddings_constraint: Constraint function applied to
 |      the `embeddings` matrix.
 |    mask_zero: Whether or not the input value 0 is a special "padding"
 |      value that should be masked out.
 |      This is useful when using recurrent layers which may take variable length input.
 |      If this is `True` then all subsequent layers in the model need to support masking or an exception will be raised.
 |      If mask_zero is set to True, as a consequence, index 0 cannot be used in the vocabulary (input_dim should equal size of vocabulary + 1).
 |    input_length: Length of input sequences, when it is constant.
 |      This argument is required if you are going to connect `Flatten` then `Dense` layers upstream (without it, the shape of the dense outputs cannot be computed).
 ```
 |||
 |---|---|
 |input_dim|단어 집합의 크기. 총 단어의 개수|
 |output_dim|임베딩 벡터의 출력차원, 결과로 나오는 임베딩 벡터의 크기|.|
 |input_length|입력 시퀀스의 길이.|

## 3.2. Sequential
인공 신경망에서 입력층, 은닉층, 출력층을 구성하기 위해 사용.  
`Sequential()`을 선언한 뒤에 `.add()`라는 코드를 통해 층을 단계적으로 추가.
~~~python
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(...) # 층 추가
model.add(...) # 층 추가
model.add(...) # 층 추가
~~~

이제 신경망을 만들어보자.

In [95]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim = len(voca)+1, output_dim = 64))
input_encoder_m.add(Dropout(rate = 0.3))

In [96]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim = len(voca)+1, output_dim = max_question_len))
input_encoder_c.add(Dropout(rate = 0.3))

In [97]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim = len(voca)+1, output_dim = 64, input_length = max_question_len))
question_encoder.add(Dropout(rate = 0.3))

In [98]:
# placeholder
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded  = question_encoder(question)